Using edited code from [Michael Wy Ong](https://medium.com/@michael.wy.ong/web-scrape-geospatial-data-analyse-singapores-property-price-part-i-276caba320b) with some tweakings

In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
## Function for getting postal code, geo coordinates of addresses

def find_postal(lst, filename):
    '''With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long)'''
    
    for index,add in enumerate(lst):
        # Do not need to change the URL
        url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add        
        print(index,url)
        
        # Retrieve information from website
        response = requests.get(url)
        try:
            data = json.loads(response.text) 
        except ValueError:
            print('JSONDecodeError')
            pass
    
        temp_df = pd.DataFrame.from_dict(data["results"])
        # The "add" is the address that was used to search in the website
        temp_df["address"] = add
        
        # Create the file with the first row that is read in 
        if index == 0:
            file = temp_df
        else:
            file = file.append(temp_df)
    file.to_csv(filename + '.csv')

In [3]:
## Function for getting closest distance of each location from a list of amenities location

from geopy.distance import geodesic

def find_nearest(house, amenity, radius=2):
    """
    this function finds the nearest locations from the 2nd table from the 1st address
    Both are dataframes with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    It also finds the number of amenities within the given radius (default=2)
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['','',100,0]
        for ind, eachloc in enumerate(amenity.iloc[:,0]):
            amenity_loc = (amenity.iloc[ind,1],amenity.iloc[ind,2])
            distance = geodesic(flat_loc,amenity_loc)
            distance = float(str(distance)[:-3]) # convert to float

            if distance <= radius:   # compute number of amenities in 2km radius
                flat_amenity[3] += 1

            if distance < flat_amenity[2]: # find nearest amenity
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance

        results[flat] = flat_amenity
    return results

In [4]:
def dist_from_location(house, location):
    """
    this function finds the distance of a location from the 1st address
    First is a dataframe with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    Second is tuple with latitude and longitude of location
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['',100]
        distance = geodesic(flat_loc,location)
        distance = float(str(distance)[:-3]) # convert to float
        flat_amenity[0] = flat
        flat_amenity[1] = distance
        results[flat] = flat_amenity
    return results


In [5]:
price1999 = pd.read_csv('Data/resale-flat-prices-based-on-approval-date-1990-1999.csv')
price2012 = pd.read_csv('Data/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
price2014 = pd.read_csv('Data/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
price2016 = pd.read_csv('Data/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')
price2017 = pd.read_csv('Data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

prices = pd.concat([price1999, price2012, price2014], sort=False)
prices = pd.concat([prices, price2016, price2017], axis=0, ignore_index=True, sort=False)

In [6]:
prices = prices[['block', 'street_name']]
prices['address'] = prices['block'] + ' ' + prices['street_name']
all_address = list(prices['address'])
unique_address = list(set(all_address))

print('Unique addresses:', len(unique_address))

Unique addresses: 9279


In [7]:
unique_address[:10]

['688D WOODLANDS DR 75',
 '511 JELAPANG RD',
 '750 PASIR RIS ST 71',
 "23A QUEEN'S CL",
 '408 YISHUN AVE 6',
 '370 HOUGANG ST 31',
 '92 BEDOK NTH AVE 4',
 '640 CHOA CHU KANG ST 64',
 '158 SIMEI RD',
 '145 LOR 2 TOA PAYOH']

In [12]:
# find_postal(unique_address, 'Data/flat_coordinates')

In [8]:
flat_coord = pd.read_csv('Data/flat_coordinates.csv')
flat_coord = flat_coord[['address','LATITUDE','LONGITUDE']]
flat_coord.head()

,address,LATITUDE,LONGITUDE
0,754 WOODLANDS CIRCLE,1.445289,103.793929
1,481 SEMBAWANG DR,1.452976,103.815094
2,506 BEDOK NTH AVE 3,1.332143,103.933720
3,688B CHOA CHU KANG DR,1.403436,103.750337
4,674 CHOA CHU KANG CRES,1.401554,103.746994


## Supermarkets

In [9]:
supermarket = pd.read_csv('Data/list-of-supermarket-licences.csv')
supermarket.head()

,licence_num,licensee_name,building_name,block_house_num,level_num,unit_num,street_name,postal_code
0,S02185J000,COLD STORAGE SINGAPORE (1983) PTE LTD,na,982,1,1,BUANGKOK CRESCENT,530982
1,S97166P000,COLD STORAGE SINGAPORE (1983) PTE LTD,HOUGANG GREEN SHOPPING MALL,21,1,52,HOUGANG STREET 51,538719
2,CE15B61K000,U STARS SUPERMARKET PTE. LTD.,na,330,1,1,ANCHORVALE STREET,540330
3,S02029J000,NTUC Fairprice Co-operative Ltd,HOUGANG POINT,1,2,1,HOUGANG STREET 91,538692
4,CE04561V000,SHENG SIONG SUPERMARKET PTE LTD,na,19,1,42401,SERANGOON NORTH AVENUE 5,554913


In [10]:
supermerket_address = list(supermarket['postal_code'])
unique_supermarket_address = list(set(supermerket_address))

print('Unique addresses:', len(unique_supermarket_address))

Unique addresses: 445


In [17]:
# find_postal(unique_supermarket_address, 'Data/supermarket_coordinates')

://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=757713
259 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=238801
260 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=768929
261 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=597071
262 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=608521
263 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=320074
264 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=380114
265 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=460018
266 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=437112
267 https://developers.onemap.sg/commonapi/sea

In [11]:
supermarket_coord = pd.read_csv('Data/supermarket_coordinates.csv')
supermarket_coord.drop_duplicates(subset=['address'], inplace=True)
supermarket_coord = supermarket_coord[['SEARCHVAL','LATITUDE','LONGITUDE']]
supermarket_coord.head()

,SEARCHVAL,LATITUDE,LONGITUDE
0,GOLDEN VILLAGE (GV KATONG),1.305225,103.905102
4,MULTI STOREY CAR PARK,1.273388,103.808981
8,11 RIVERVALE CRESCENT SINGAPORE 545082,1.392504,103.904484
14,CORALINUS,1.403719,103.906138
16,WESTLITE JUNIPER,1.406303,103.760615


In [12]:
nearest_supermarket = find_nearest(flat_coord, supermarket_coord)
flat_supermarket = pd.DataFrame.from_dict(nearest_supermarket).T
flat_supermarket = flat_supermarket.rename(columns={0: 'flat', 1: 'supermarket', 2: 'supermarket_dist', 3: 'num_supermarket_2km'}).reset_index().drop(['index','supermarket'], axis=1)
flat_supermarket.head()

,flat,supermarket_dist,num_supermarket_2km
0,754 WOODLANDS CIRCLE,0.430366,14
1,481 SEMBAWANG DR,0.269066,14
2,506 BEDOK NTH AVE 3,0.296136,21
3,688B CHOA CHU KANG DR,0.798057,6
4,674 CHOA CHU KANG CRES,0.472646,7


In [13]:
flat_supermarket.to_csv('Data/flat_supermarket.csv', index=False)

## Schools

In [14]:
school = pd.read_csv('Data/school_names.csv', encoding='cp1252')
school.head(10)

,school,area
0,Admiralty Primary School,Woodlands
1,Ahmad Ibrahim Primary School,Yishun
2,Ai Tong School,Bishan
3,Alexandra Primary School,Bukit Merah
4,Anchor Green Primary School,Sengkang
5,Anderson Primary School,Ang Mo Kio
6,Anglo-Chinese School (Junior),Central
7,Anglo-Chinese School (Primary),Bukit Timah
8,Angsana Primary School,Tampines
9,Ang Mo Kio Primary School,Ang Mo Kio


In [15]:
school_name = list(school['school'])
unique_school_name = list(set(school_name))

print('Unique addresses:', len(unique_school_name))

Unique addresses: 191


In [33]:
# find_postal(unique_school_name, 'Data/school_coordinates')

berra Primary School
31 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Riverside Primary School
32 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Meridian Primary School
33 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Rosyth School
34 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Cantonment Primary School
35 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Westwood Primary School
36 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Zhangde Primary School
37 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Fuhua Primary School
38 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Concord Primary School
39 http

In [16]:
school_coord = pd.read_csv('Data/school_coordinates.csv')
school_coord = school_coord[['address','LATITUDE','LONGITUDE']]
school_coord.head()

,address,LATITUDE,LONGITUDE
0,Princess Elizabeth Primary School,1.349195,103.741000
1,Woodgrove Primary School,1.433058,103.790390
2,Nanyang Primary School,1.321115,103.806468
3,Nanyang Primary School,1.321246,103.807795
4,Waterway Primary School,1.399154,103.918671


In [17]:
nearest_school = find_nearest(flat_coord, school_coord)
flat_school = pd.DataFrame.from_dict(nearest_school).T
flat_school = flat_school.rename(columns={0: 'flat', 1: 'school', 2: 'school_dist', 3: 'num_school_2km'}).reset_index().drop('index', axis=1)
flat_school.head()

,flat,school,school_dist,num_school_2km
0,754 WOODLANDS CIRCLE,Evergreen Primary School,0.14358,10
1,481 SEMBAWANG DR,Canberra Primary School,0.232877,7
2,506 BEDOK NTH AVE 3,Red Swastika School,0.16621,8
3,688B CHOA CHU KANG DR,Unity Primary School,0.3782,4
4,674 CHOA CHU KANG CRES,Unity Primary School,0.140389,4


In [20]:
flat_school.to_csv('Data/flat_school.csv', index=False)

## Hawker and Markets

In [21]:
hawker = pd.read_csv('Data/list-of-government-markets-hawker-centres.csv')
hawker.head(10)

,name_of_centre,location_of_centre,type_of_centre,owner,no_of_stalls,no_of_cooked_food_stalls,no_of_mkt_produce_stalls
0,Adam Road Food Centre,"2, Adam Road, S(289876)",HC,Government,32,32,0
1,Amoy Street Food Centre,"National Development Building, Annex B, Telok ...",HC,Government,135,134,1
2,Bedok Food Centre,"1, Bedok Road, S(469572)",HC,Government,32,32,0
3,Beo Crescent Market,"38A, Beo Crescent, S(169982)",MHC,Government,94,32,62
4,Berseh Food Centre,"166, Jalan Besar, S(208877)",HC,Government,66,66,0
5,Bukit Timah Market,"51, Upper Bukit Timah Road, S(588215)",MHC,Government,179,84,95
6,Chomp Chomp Food Centre,"20, Kensington Park Road, S(557269)",HC,Government,36,36,0
7,Commonwealth Crescent Market,"31, Commonwealth Crescent, S(149644)",MHC,Government,102,39,63
8,Dunman Food Centre,"271, Onan Road, S(424768)",HC,Government,30,30,0
9,East Coast Lagoon Food Village,"1220, East Coast Parkway, S(468960)",HC,Government,63,63,0


In [22]:
hawker_name = list(hawker['name_of_centre'])
unique_hawker_name = list(set(hawker_name))

print('Unique addresses:', len(unique_hawker_name))

Unique addresses: 107


In [38]:
# find_postal(unique_hawker_name, 'Data/hawker_coordinates')

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 82 Telok Blangah Drive
1 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 503 West Coast Drive
2 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Amoy Street Food Centre
3 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blks 20/21 Marsiling Lane
4 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 36 Telok Blangah Rise
5 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blks 91/92 Whampoa Drive
6 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Geylang Serai Market
7 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blks 2 & 3 Changi Village Road
8 https://d

In [23]:
hawker_coord = pd.read_csv('Data/hawker_coordinates.csv')
hawker_coord = hawker_coord[['address','LATITUDE','LONGITUDE']]
hawker_coord.head()

,address,LATITUDE,LONGITUDE
0,Amoy Street Food Centre,1.279340,103.846653
1,Blks 20/21 Marsiling Lane,1.443444,103.776989
2,Blks 20/21 Marsiling Lane,1.443832,103.777210
3,Geylang Serai Market,1.316725,103.897977
4,Blks 2 & 3 Changi Village Road,1.443444,103.776989


In [24]:
nearest_hawker = find_nearest(flat_coord, hawker_coord)
flat_hawker = pd.DataFrame.from_dict(nearest_hawker).T
flat_hawker = flat_hawker.rename(columns={0: 'flat', 1: 'hawker', 2: 'hawker_dist', 3: 'num_hawker_2km'}).reset_index().drop('index', axis=1)
flat_hawker.head()

,flat,hawker,hawker_dist,num_hawker_2km
0,754 WOODLANDS CIRCLE,Blks 20/21 Marsiling Lane,1.867462,4
1,481 SEMBAWANG DR,Chong Pang Market & Food Centre,2.753292,0
2,506 BEDOK NTH AVE 3,Market Street Food Centre,0.360096,8
3,688B CHOA CHU KANG DR,Blks 20/21 Marsiling Lane,5.326198,0
4,674 CHOA CHU KANG CRES,Blks 20/21 Marsiling Lane,5.70949,0


In [25]:
flat_hawker.to_csv('Data/flat_hawker.csv', index=False)

## Shopping Malls

In [27]:
# shop = pd.read_csv('Data/shoppingmalls.csv', encoding='cp1252')
# shop.head()

In [58]:
shop_name = list(shop['name'])
unique_shop_name = list(set(shop_name))

print('Unique addresses:', len(unique_shop_name))

Unique addresses: 173


In [60]:
# find_postal(unique_shop_name, 'Data/shoppingmall_coordinates')

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Katong Square
1 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=PLQ Mall
2 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Capitol Piazza
3 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Bugis Cube
4 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=HillV2
5 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=100 AM
6 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Midpoint Orchard
7 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Woodlands Mart
8 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Bukit Timah Plaza
9 htt

In [29]:
shop_coord = pd.read_csv('Data/shoppingmall_coordinates_clean.csv')
shop_coord.drop_duplicates(subset=['address'], inplace=True)
shop_coord = shop_coord[['address','LATITUDE','LONGITUDE']]
shop_coord.head()

,address,LATITUDE,LONGITUDE
0,Katong Square,1.304853,103.904574
1,PLQ Mall,1.317482,103.892832
2,Capitol Piazza,1.293063,103.851293
3,Bugis Cube,1.298195,103.855655
4,HillV2,1.363551,103.764236


In [30]:
nearest_mall = find_nearest(flat_coord, shop_coord)
flat_mall = pd.DataFrame.from_dict(nearest_mall).T
flat_mall = flat_mall.rename(columns={0: 'flat', 1: 'mall', 2: 'mall_dist', 3: 'num_mall_2km'}).reset_index().drop('index', axis=1)
flat_mall.head()

,flat,mall,mall_dist,num_mall_2km
0,754 WOODLANDS CIRCLE,Woodlands Mart,0.47279,4
1,481 SEMBAWANG DR,Sun Plaza,0.715626,2
2,506 BEDOK NTH AVE 3,Djitsun Mall,0.814369,5
3,688B CHOA CHU KANG DR,Yew Tee Square,0.697144,3
4,674 CHOA CHU KANG CRES,Yew Tee Square,0.387852,4


In [31]:
flat_mall.to_csv('Data/flat_mall.csv', index=False)

## Parks

In [32]:
park_coord = pd.read_csv('Data/parks_coordinates_clean.csv')
park_coord.reset_index(inplace=True)
park_coord = park_coord[['index','Y','X']]
park_coord.head()

,index,Y,X
0,0,1.346175,103.960857
1,1,1.389904,103.978979
2,2,1.409913,103.923131
3,3,1.379237,103.866967
4,4,1.372761,103.829073


In [33]:
nearest_park = find_nearest(flat_coord, park_coord)
flat_park = pd.DataFrame.from_dict(nearest_park).T
flat_park = flat_park.rename(columns={0: 'flat', 1: 'park', 2: 'park_dist', 3: 'num_park_2km'}).reset_index().drop(['index','park'], axis=1)
flat_park.head()

,flat,park_dist,num_park_2km
0,754 WOODLANDS CIRCLE,1.326709,3
1,481 SEMBAWANG DR,1.622653,1
2,506 BEDOK NTH AVE 3,1.08729,19
3,688B CHOA CHU KANG DR,1.540195,3
4,674 CHOA CHU KANG CRES,1.382071,2


In [34]:
flat_park.to_csv('Data/flat_park.csv', index=False)

## MRT

In [35]:
mrt_coord = pd.read_csv('Data/MRT_coordinates.csv')
mrt_coord = mrt_coord[['STN_NAME','Latitude','Longitude']]
mrt_coord.head()

,STN_NAME,Latitude,Longitude
0,ADMIRALTY MRT STATION,1.440585,103.800998
1,ALJUNIED MRT STATION,1.316433,103.882893
2,ANG MO KIO MRT STATION,1.369933,103.849553
3,BAKAU LRT STATION,1.388093,103.905418
4,BANGKIT LRT STATION,1.380018,103.772667


In [36]:
nearest_mrt = find_nearest(flat_coord, mrt_coord)
flat_mrt = pd.DataFrame.from_dict(nearest_mrt).T
flat_mrt = flat_mrt.rename(columns={0: 'flat', 1: 'mrt', 2: 'mrt_dist', 3: 'num_mrt_2km'}).reset_index().drop('index', axis=1)
flat_mrt.head()

,flat,mrt,mrt_dist,num_mrt_2km
0,754 WOODLANDS CIRCLE,ADMIRALTY MRT STATION,0.943118,4
1,481 SEMBAWANG DR,SEMBAWANG MRT STATION,0.70187,1
2,506 BEDOK NTH AVE 3,BEDOK RESERVOIR MRT STATION,0.521625,5
3,688B CHOA CHU KANG DR,YEW TEE MRT STATION,0.728251,1
4,674 CHOA CHU KANG CRES,YEW TEE MRT STATION,0.447021,3


In [37]:
flat_mrt.to_csv('Data/flat_mrt.csv', index=False)

In [38]:
flat_coord = pd.read_csv('Data/flat_coordinates_clean.csv')
flat_coord = flat_coord[['address','LATITUDE','LONGITUDE']]
flat_coord.head()

,address,LATITUDE,LONGITUDE
0,754 WOODLANDS CIRCLE,1.445289,103.793929
1,481 SEMBAWANG DR,1.452976,103.815094
2,506 BEDOK NTH AVE 3,1.332143,103.933720
3,688B CHOA CHU KANG DR,1.403436,103.750337
4,674 CHOA CHU KANG CRES,1.401554,103.746994


## Merge All

In [39]:
## Merge all

flat_amenities = flat_school.merge(flat_hawker, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_park, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_mall, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_mrt, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_supermarket, on='flat', how='outer')

flat_amenities.head()

,flat,school,school_dist,num_school_2km,hawker,hawker_dist,num_hawker_2km,park_dist,num_park_2km,mall,mall_dist,num_mall_2km,mrt,mrt_dist,num_mrt_2km,supermarket_dist,num_supermarket_2km
0,754 WOODLANDS CIRCLE,Evergreen Primary School,0.14358,10,Blks 20/21 Marsiling Lane,1.867462,4,1.326709,3,Woodlands Mart,0.47279,4,ADMIRALTY MRT STATION,0.943118,4,0.430366,14
1,481 SEMBAWANG DR,Canberra Primary School,0.232877,7,Chong Pang Market & Food Centre,2.753292,0,1.622653,1,Sun Plaza,0.715626,2,SEMBAWANG MRT STATION,0.70187,1,0.269066,14
2,506 BEDOK NTH AVE 3,Red Swastika School,0.16621,8,Market Street Food Centre,0.360096,8,1.08729,19,Djitsun Mall,0.814369,5,BEDOK RESERVOIR MRT STATION,0.521625,5,0.296136,21
3,688B CHOA CHU KANG DR,Unity Primary School,0.3782,4,Blks 20/21 Marsiling Lane,5.326198,0,1.540195,3,Yew Tee Square,0.697144,3,YEW TEE MRT STATION,0.728251,1,0.798057,6
4,674 CHOA CHU KANG CRES,Unity Primary School,0.140389,4,Blks 20/21 Marsiling Lane,5.70949,0,1.382071,2,Yew Tee Square,0.387852,4,YEW TEE MRT STATION,0.447021,3,0.472646,7


In [40]:
flat_amenities.to_csv('Data/flat_amenities.csv', index=False)

## Get dist from dhoby ghaut

In [41]:
flat_coord = pd.read_csv('Data/flat_coordinates.csv')
flat_coord = flat_coord[['address','LATITUDE','LONGITUDE']]

dist_dhoby = dist_from_location(flat_coord, (1.299308, 103.845285))
dist_dhoby = pd.DataFrame.from_dict(dist_dhoby).T
dist_dhoby = dist_dhoby.rename(columns={0: 'flat', 1: 'dist_dhoby'}).reset_index().drop(['index'], axis=1)
dist_dhoby.head()

,flat,dist_dhoby
0,754 WOODLANDS CIRCLE,17.123778
1,481 SEMBAWANG DR,17.320854
2,506 BEDOK NTH AVE 3,10.490312
3,688B CHOA CHU KANG DR,15.627695
4,674 CHOA CHU KANG CRES,15.73137


In [42]:
flat_amenities = pd.read_csv('Data/flat_amenities.csv')
flat_amenities = flat_amenities.merge(dist_dhoby, on='flat', how='outer')
flat_amenities.head()

,flat,school,school_dist,num_school_2km,hawker,hawker_dist,num_hawker_2km,park_dist,num_park_2km,mall,mall_dist,num_mall_2km,mrt,mrt_dist,num_mrt_2km,supermarket_dist,num_supermarket_2km,dist_dhoby
0,754 WOODLANDS CIRCLE,Evergreen Primary School,0.143580,10,Blks 20/21 Marsiling Lane,1.867462,4,1.326709,3,Woodlands Mart,0.472790,4,ADMIRALTY MRT STATION,0.943118,4,0.430366,14,17.123778
1,481 SEMBAWANG DR,Canberra Primary School,0.232877,7,Chong Pang Market & Food Centre,2.753292,0,1.622653,1,Sun Plaza,0.715626,2,SEMBAWANG MRT STATION,0.701870,1,0.269066,14,17.320854
2,506 BEDOK NTH AVE 3,Red Swastika School,0.166210,8,Market Street Food Centre,0.360096,8,1.087290,19,Djitsun Mall,0.814369,5,BEDOK RESERVOIR MRT STATION,0.521625,5,0.296136,21,10.490312
3,688B CHOA CHU KANG DR,Unity Primary School,0.378200,4,Blks 20/21 Marsiling Lane,5.326198,0,1.540195,3,Yew Tee Square,0.697144,3,YEW TEE MRT STATION,0.728251,1,0.798057,6,15.627695
4,674 CHOA CHU KANG CRES,Unity Primary School,0.140389,4,Blks 20/21 Marsiling Lane,5.709490,0,1.382071,2,Yew Tee Square,0.387852,4,YEW TEE MRT STATION,0.447021,3,0.472646,7,15.73137


In [43]:
flat_amenities.to_csv('Data/flat_amenities.csv', index=False)